In [2]:
import numpy as np 
# linear algebra 
import pandas as pd 
import matplotlib.pyplot as plt 
# for data visualization purposes 
import seaborn as sns 
# for statistical data visualization 
%matplotlib inline 

In [3]:
df = pd.read_csv('Desktop/New/Projects/2. XGBoost/bile_buyers_clean.csv')
df.head()

FileNotFoundError: [Errno 2] File Desktop/New/Projects/2. XGBoost/bile_buyers_clean.csv does not exist: 'Desktop/New/Projects/2. XGBoost/bile_buyers_clean.csv'

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
corrMatrix = df.corr() sns.heatmap(corrMatrix, annot=True) plt.show() 

### Analyzing Numerical Variables 

In [ ]:
numerical = [var for var in df.columns if df[var].dtype!='O'] 
print('There are {} numerical variables'.format(len(numerical))) 
print('The numerical variables are :', numerical)

In [ ]:
df[numerical].head() 


In [ ]:
# check missing values in numerical variables 
df[numerical].isnull().sum() 

### Analyzing Categorical Variables

In [ ]:
categorical = [var for var in df.columns if df[var].dtype=='O'] 
print('There are {} categorical variables'.format(len(categorical))) 
print('The categorical variables are :', categorical)

In [ ]:
df[categorical].head() 

In [ ]:
df[categorical].isnull().sum()

In [ ]:

# view frequency counts of values in categorical variables 
for var in categorical: 
    print(df[var].value_counts()) 
    print(df[var].value_counts()/np.float(len(df))) 
    print() 

M

In [ ]:
# check for cardinality in categorical variables 
for var in categorical: 
    print(var, ' contains ', len(df[var].unique()), ' labels')

### Label Encoding

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df['Marital Status'] = label_encoder.fit_transform(df['Marital Status']) 
df['Gender'] = label_encoder.fit_transform(df['Gender']) 
df['Education'] = label_encoder.fit_transform(df['Education']) 
df['Occupation'] = label_encoder.fit_transform(df['Occupation']) 
df['Home Owner'] = label_encoder.fit_transform(df['Home Owner']) 
df['Commute Distance'] = label_encoder.fit_transform(df['Commute Distance']) 
df['Region'] = label_encoder.fit_transform(df['Region']) 
df['Purchased Bike'] = label_encoder.fit_transform(df['Purchased Bike']) 
df.head() 

### Categorize Continuous Data

In [ ]:
df['Age'].describe()

In [ ]:
df['Age'] = pd.cut(x = df['Age'], bins = [0,30,40,50,60,100,150], labels = [0, 1, 2, 3, 4, 5]) 
df['Age'] = df['Age'].astype('int64') 
df['Age'].isnull().sum() 

In [ ]:
df['Income'].describe()

In [ ]:
df['Income'] = pd.cut(x = df['Income'], bins = [0, 30000, 50000, 75000, 100000, 150000, 200000], labels = [1, 2, 3, 4, 5, 6]) 
df['Income'] = df['Income'].astype('int64') 
df['Income'].isnull().sum() 

In [ ]:
df.dtypes

### Train_Test_Split

In [ ]:
X = df.drop(['Purchased Bike'])
y = df['Purchased Bike']

In [ ]:
# split X and y into training and testing sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 999) 
X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape()

### Gausian Naive Bayes

In [ ]:
# train a Gaussian Naive Bayes classifier on the training set 
from sklearn.naive_bayes import GaussianNB 
# instantiate the model 
gnb = GaussianNB() 
# fit the model 
gnb.fit(X_train, y_train)

In [ ]:
y_pred = gnb.predict(X_test) 
y_pred[:10] 
len(y_pred) 

In [ ]:
from sklearn.metrics import accuracy_score 
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
y_pred_train = gnb.predict(X_train) 
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train))) 

In [ ]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred) 
print('Confusion matrix\n', cm) 

In [ ]:
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive', 'Actual Negative'], index=['Predict Positive', 'Predict Negative']) 
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu') 
plt.show() 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics 
# Create Decision Tree classifer object 
clf = DecisionTreeClassifier(criterion="gini", max_depth=3, random_state=999) 
# Train Decision Tree Classifer 
clf = clf.fit(X_train,y_train) 
# Predict the response for test dataset 
y_pred2 = clf.predict(X_test) 

In [ ]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred2))) 

In [ ]:
y_pred_train2 = clf.predict(X_train) 
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train2))) 

### Visualize Decision Tree

In [ ]:
from sklearn import tree 
fn=['ID', 'Marital Status', 'Gender', 'Income', 'Children', 'Education', 'Occupation', 'Home Owner', 'Cars', 'Commute Distance', 'Region', 'Age'] cn=['Bought', 'Not Bought'] 
fig = plt.figure(figsize=(25,20)) 
_ = tree.plot_tree(clf, feature_names = fn, class_names=cn, filled=True) 

### Improving Accuracy using Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV 
clf = DecisionTreeClassifier(criterion="gini", max_depth=3) 
grid_values = {'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth':[4,5,6,7,8,9,10], 'min_samples_split': [2,3,4]} 
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values) grid_clf_acc.fit(X_train, y_train)

In [ ]:
y_pred_acc = grid_clf_acc.predict(X_test) 
# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_acc))) 
#Confusion matrix 
cm = confusion_matrix(y_test,y_pred_acc) 
print(cm) 

In [ ]:
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive', 'Actual Negative'], index=['Predict Positive', 'Predict Negative']) 
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu') 
plt.show() 

In [ ]:
from sklearn.metrics import classification_report 
print(classification_report(y_test, y_pred2)) 

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier() 
model.fit(X_train, y_train) 
result = model.score(X_test, y_test) 
print('Model accuracy score: {0:0.4f}'. format(result)) 

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score 
rfc = RandomForestClassifier() 
# Choose some parameter combinations to try 
parameters = {'n_estimators': [4, 6, 9], 'max_features': ['log2', 'sqrt','auto'], 'criterion': ['entropy', 'gini'], 'max_depth': [2, 3, 5, 10], 'min_samples_split': [2, 3, 5], 'min_samples_leaf': [1,5,8] } 
# Run the grid search 
grid_obj = GridSearchCV(rfc, parameters) 
grid_obj = grid_obj.fit(X_train, y_train) 
# Set the clf to the best combination of parameters 
rfc = grid_obj.best_estimator_ 
# Fit the best algorithm to the data 
rfc.fit(X_train, y_train) 

In [ ]:
y_pred4 = grid_clf_acc.predict(X_test) 
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred4))) 

### XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier 
model = XGBClassifier() 
model.fit(X_train, y_train) 
y_pred = model.predict(X_test) 
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Final Note

In this kernel, I have analysed the Bike Buyers dataset and performed Binary Classification using various Supervised Learning Classification algorithms. The accuracy is fairly less due the the limitations of the dataset. 